# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json
import gmaps
# import json
import os

# Import API key
from config import key

In [2]:
# Load city data from WeatherPy
city_data = "../WeatherPy/Output/city_data.csv"
city_data_df = pd.read_csv(city_data, index_col=[0], encoding="ISO-8859-1")
city_data_df.head()
# city_data_df.dtypes

,City_Name,City_Lat,City_Long,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date
0,MahÃ©bourg,-20.4081,57.7000,70.09,94,75,9.22,MU,2021-06-17 17:59:38
1,Oranjestad,12.5240,-70.0270,89.55,62,20,21.85,AW,2021-06-17 17:59:39
2,Grand Gaube,-20.0064,57.6608,70.38,84,14,1.99,MU,2021-06-17 17:59:39
3,Hasaki,35.7333,140.8333,62.98,80,10,11.34,JP,2021-06-17 17:51:44
4,Puerto Ayora,-0.7393,-90.3518,78.55,83,75,5.99,EC,2021-06-17 17:51:45


In [3]:
# Rename columns and create new dataframe with dropped NA values 
city_data_df = city_data_df.rename(columns={'City_Name': 'City', 'City_Lat': 'Latitude', 
                                            'City_Long': 'Longitude', 'Wind_Speed': 'Wind Speed'})

city_df_clean = city_data_df.dropna(axis = 0, how ='any')

# comparing sizes of data frames
print("Old data frame length:", len(city_data_df), "\nNew data frame length:", 
       len(city_df_clean), "\nNumber of rows with at least 1 NA value: ",
       (len(city_data_df)-len(city_df_clean)))
city_df_clean.to_csv("./output_data/city_df_clean.csv")

Old data frame length: 645 
New data frame length: 639 
Number of rows with at least 1 NA value:  6


### Humidity Heatmap

In [4]:
# Configure gmaps
gmaps.configure(api_key=key)

# Set Locations for the heatmap
locations = city_df_clean[["Latitude", "Longitude"]]

humidity = city_df_clean["Humidity"].astype(float)

In [ ]:
# Plot humidity heatmap
fig = gmaps.figure()

# Create humidity heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=3)
# Add layer and display fig
fig.add_layer(heat_layer)
fig

### Ideal Weather Conditions

In [5]:
# Create new dataframe based on indicated parameters
ideal_cities = city_df_clean[(city_df_clean['Temperature']>80) & (city_df_clean['Temperature']<90) 
                   & (city_df_clean['Humidity']>40) & (city_df_clean['Humidity']<70) 
                   & (city_df_clean['Wind Speed']<10) & (city_df_clean['Cloudiness']<20)]
ideal_cities.head()
# len(ideal_cities)    #---- check number of cities to meet rubric criteria; uncomment to enable 

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
13,Rome,34.2570,-85.1647,84.45,42,1,1.99,US,2021-06-17 17:57:41
79,Hamilton,39.1834,-84.5333,83.10,44,19,1.99,US,2021-06-17 18:00:10
99,Russell,32.3502,-85.1999,88.86,42,0,4.50,US,2021-06-17 18:00:06
260,Prainha,-1.8000,-53.4800,87.44,55,13,4.03,BR,2021-06-17 18:01:38
271,Havelock,34.8791,-76.9013,82.76,61,0,4.00,US,2021-06-17 18:01:43


### Find Nearest Hotel

In [11]:
# Create new dataframe by method drop
hotels_df = ideal_cities.drop(['Temperature', 'Humidity', 'Cloudiness', 'Wind Speed'], axis=1)

# Add column to hotels dataframe
hotels_df['Hotel Name'] = ""
hotels_df.head()

,City,Latitude,Longitude,Country,Date,Hotel Name
13,Rome,34.2570,-85.1647,US,2021-06-17 17:57:41,
79,Hamilton,39.1834,-84.5333,US,2021-06-17 18:00:10,
99,Russell,32.3502,-85.1999,US,2021-06-17 18:00:06,
260,Prainha,-1.8000,-53.4800,BR,2021-06-17 18:01:38,
271,Havelock,34.8791,-76.9013,US,2021-06-17 18:01:43,


In [ ]:
# Parameters dictionary for Nearby search requests
params = {
#     'location': 'city_geo',
    'types': 'lodging', # keyword
    'radius': 5000, 
    'key': key
}
# Nearby search base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iterate through pandas dataframe
for i, row in hotels_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    params['location'] = f'{lat},{long}'

    try:
        # Construct url and make API request
        response = requests.get(base_url, params)
        json_results = response.json()
        hotel_name = json_results['results'][0]['name']
        hotels_df['Hotel Name'][i] = hotel_name      

    except:
        print('Missing field for city of {}...skipping'.format(row['City']))
        print('-'*26)

hotels_df.to_csv('./output_data/hotels_df.csv')
hotels_df.head()